# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features['B']
logdis = np.log(boston_features['DIS'])
loglstat = np.log(boston_features['LSTAT'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns = ['target'])
type(X)

pandas.core.frame.DataFrame

### Train-test split

Perform a train-test split with a test set of 20%.

In [7]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [8]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20)

In [9]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [12]:
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)
print(y_hat_train)
print(y_hat_test)

[[27.0813344 ]
 [21.22839814]
 [20.12086521]
 [18.7593576 ]
 [20.12625682]
 [20.96238691]
 [36.43858409]
 [20.23514532]
 [27.13659643]
 [27.06390666]
 [18.71375805]
 [28.53904697]
 [32.71037038]
 [14.30721367]
 [40.588913  ]
 [22.40823921]
 [25.89961204]
 [23.84650707]
 [19.4765796 ]
 [42.37545315]
 [31.33389938]
 [24.51573274]
 [23.36789496]
 [18.10268102]
 [21.07485817]
 [33.692964  ]
 [14.65173203]
 [31.66296562]
 [25.64461709]
 [15.9997815 ]
 [45.16656652]
 [27.98813782]
 [23.06892339]
 [19.79640536]
 [16.77176273]
 [11.40101111]
 [18.64291006]
 [22.53237473]
 [21.30784593]
 [17.18142695]
 [17.73578946]
 [17.53443964]
 [14.42929105]
 [34.57562732]
 [29.08431015]
 [20.87730833]
 [14.81656968]
 [23.31339648]
 [19.80066802]
 [30.3303003 ]
 [19.99734204]
 [26.56161939]
 [23.46424591]
 [28.70264819]
 [18.78961241]
 [24.97485044]
 [38.19295478]
 [21.14396406]
 [26.08608569]
 [24.55216443]
 [27.15143901]
 [26.84205766]
 [ 7.88583419]
 [26.25766171]
 [20.8465666 ]
 [ 8.62957775]
 [16.05666

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [13]:
from sklearn.metrics import mean_squared_error
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squared Error: ', train_mse)
print('Test Mean Squared Error: ', test_mse)

Train Mean Squared Error:  17.833400965941816
Test Mean Squared Error:  37.41015992295277


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [14]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_obs = 0
    # add 1 to fold size to account for leftovers  
    for fold_n in range(1, k+1):
        if fold_n <- leftovers:
            fold = data.iloc[start_obs: start_obs+fold_size+1]
            folds.append(fold)
            start_obs += fold_size + 1
        else:
            fold = data.iloc[start_obs: start_obs+fold_size]
            folds.append(fold)
            start_obs += fold_size
    return folds

### Apply it to the Boston Housing data

In [15]:
# Make sure to concatenate the data again
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [16]:
# Apply kfolds() to bos_data with 5 folds
bos_folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [17]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    # Evaluate Train and Test errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals =y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))

print(train_errs)
print(test_errs)

[target    23.663952
dtype: float64, target    22.67354
dtype: float64, target    19.23146
dtype: float64, target    14.98836
dtype: float64, target    22.34583
dtype: float64]
[target    13.758664
dtype: float64, target    17.017538
dtype: float64, target    37.586711
dtype: float64, target    57.490691
dtype: float64, target    24.738277
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [21]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
cv_5_results  = (cross_val_score(linreg, X, y, cv=5,  scoring='neg_mean_squared_error'))
cv_5_results

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [22]:
cv_5_results_mean  = np.mean(cross_val_score(linreg, X, y, cv=5,  scoring='neg_mean_squared_error'))
cv_5_results_mean

-30.451881143540298

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 